In [37]:
#libraries
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import pandas as pd
import re

In [28]:
url='https://www.emag.ro/monitoare-lcd-led/c?ref=hp_menu_quick-nav_23_3&type=category'
HEADERS = {    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 OPR/112.0.0.0"}


In [38]:
pagina_monitoare = requests.get(url, headers = HEADERS)

In [40]:
pagina_monitoare

<Response [511]>

In [31]:
soup = BeautifulSoup(pagina_monitoare.content , 'html.parser')
soup

<!DOCTYPE html>
<html class="lang_ro" lang="ro"><head><link href="https://www.emag.ro/" rel="canonical"/><meta charset="utf-8"/><meta content="width=device-width, height=device-height, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"><link href="//s13emagst.akamaized.net/assets/ro/css/icons/favicon.ico?v=2b" rel="shortcut icon"/><title>eMAG.ro</title> <script async="" defer="" src="https://challenges.cloudflare.com/turnstile/v0/api.js?compat=recaptcha&amp;hl=ro&amp;onload=lcb&amp;render=explicit"></script> <style>body{background-color:#FFF;font-family:'Open Sans',Helvetica,Arial,sans-serif;font-size:1em}.captcha-widget,.emg-button{display:inline-block;position:relative;overflow:hidden;box-sizing:border-box}.m-title-container .emg-button{max-width:180px;text-decoration:none}.wrapper{max-width:315px;margin:0 auto}.wrapper-h-captcha .wrapper{max-width:350px}.captcha-widget{padding:20px;border:none;box-shadow:0px 0px 5px rgba(0,0,0,0.1);border-radius:12px;margin-botto

In [32]:
def find_monitor_links(soup):
    links = soup.find_all('a')
    link_list = []
    for link in links:
        href = link.get('href')
        if href and 'https://www.emag.ro/monitor-' in href :
            link_list.append(href)

    return link_list




In [33]:
link_list = set(find_monitor_links(soup))


In [34]:
len(link_list)

0

In [36]:

possible_resolutions = []
resolution_box = soup.find('div',id = 'js-filter-9839-collapse')
resolution_box =  resolution_box.find_all('a')
print(resolution_box)
for resolution in resolution_box:
    possible_resolutions.append(resolution.text.splitlines())
possible_resolutions
cleaned_list = []
for item in possible_resolutions:
    for elem in item:
        cleaned_list.append(elem)



AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:

resolution_pattern = r'\d{3,4}\s*[xX×]\s*\d{3,4}'
matching_patterns = [item for item in cleaned_list if re.match(resolution_pattern,item)]
print(sorted(matching_patterns))

In [ ]:
non_qm_link_list = []



for link in link_list:
    if link[-1] != '?':
       
        non_qm_link_list.append(link)
      

In [ ]:
print('\n'.join(non_qm_link_list))

In [ ]:
len(non_qm_link_list)

In [ ]:
produs_de_test = requests.get(non_qm_link_list[0], headers = HEADERS)
produs_de_test


In [ ]:
test_soup = BeautifulSoup(produs_de_test.content , 'html.parser')
import difflib

tabel_specificatii = test_soup.find_all('td' , class_ ="col-xs-8")
specificatii = [specificatie.text.strip() for specificatie in tabel_specificatii]


def find_most_similar_pair(list1, list2):
    best_pair = (None, None)
    best_similarity = 0.0
    
    for item1 in list1:
        for item2 in list2:
            # Calculate the similarity ratio between the two items
            similarity = difflib.SequenceMatcher(None, item1, item2).ratio()
            
            # If this pair has a higher similarity score, update best_pair
            if similarity > best_similarity:
                best_pair = (item1, item2)
                best_similarity = similarity
                
    return best_pair

# Get the most similar pair and its similarity score
most_similar_pair = find_most_similar_pair(specificatii,matching_patterns)
most_similar_pair[0]

pattern_refresh_rate = r"\d+\s*[Hh][Zz]"
refresh_rate = [re.findall(pattern_refresh_rate,specificatie) for specificatie in specificatii]
refresh_rate = [item for sublist in refresh_rate for item in sublist]
print('the refresh rate is ')
print(refresh_rate)
specificatii

pattern = r'(\d+\.?\d*)\s?(inch)'


dimension = [re.findall(pattern,specificatie) for specificatie in specificatii]
dimension = [item for sublist in dimension for item in sublist]
print('the dimension is ')
print(dimension)
dimension = dimension[0][0] + ' ' +  dimension[0][1]
dimension



#cod produs
cod_produs  = test_soup.find('span' , class_="product-code-display hidden-xs")
cod_produs.text.strip().split()[2]

In [ ]:
coloane = ['Nume' , 'Firma'  , 'Pret' , 'Dimensiune (Inci)' , 'Rezolutie' , 'Refresh rate' , 'Rating', 'Nr Review', 'Cod produs']

In [ ]:
header = pd.DataFrame(columns = coloane)
header

## Get Monitor Information

In [ ]:
def get_monitor_information(monitor):
    monitor_details = []

    #get the name of the monitor
    product_name = monitor.find('title').text
    if ' - eMAG.ro' in product_name: product_name = product_name.removesuffix(' - eMAG.ro')
    monitor_details.append(product_name)
    print("The name of the monitor was added")
    print(monitor_details)

    #get the name of the brand/firm
    brand = 'Brand: '
    p_tags = monitor.find_all('p')

   
    for p in p_tags:
        if 'Brand:' in p.text:
            # Extract the brand name from the <a> tag
            brand_name = p.find('a').text
            break
    if brand_name:
        monitor_details.append(brand_name)
        print("The name of the brand was added")
        print(monitor_details)
    else: 
        brand_name = 'not found'
        monitor_details.append(brand_name)
        print("The name of the brand was not found")
        print(monitor_details)



    #getting the price
    
    price = monitor.find('p' , class_ = 'product-new-price' )
    
    if price:
        price = price.text
        monitor_details.append(price)
        print("The price of the monitor was added")
        print(monitor_details)
    else:
        price = 'not found'
        monitor_details.append(price)
        print("The price was not found")
        print(monitor_details)
        
    

    #getting the dimension
    
    tabel_specificatii = monitor.find_all('td' , class_ ="col-xs-8")
    specificatii = [specificatie.text.strip() for specificatie in tabel_specificatii]
    pattern = r'(\d+\.?\d*)\s?(inch)'


    dimension = [re.findall(pattern,specificatie) for specificatie in specificatii]
    dimension = [item for sublist in dimension for item in sublist]
    if dimension:
        print('the dimension is ')
        print(dimension)
        dimension = dimension[0][0] + ' ' +  dimension[0][1]
 
    
    if dimension:
            monitor_details.append(dimension)
            print('the dimension was added')
            print(monitor_details)  
    else: 
            dimension = 'not found'
            monitor_details.append(dimension)
            print('the dimension could not be found')
            print(monitor_details) 
            


    #getting the resolutiion

    
    rezolutie_pereche = find_most_similar_pair(specificatii,matching_patterns)
    rezolutie = rezolutie_pereche[0]
  
    if rezolutie:
        monitor_details.append(rezolutie)
        print('Added resolution')
        print(monitor_details) 
    else: 
        monitor_details.append('not found')
        print('the refresh_rate could not be found')
        print(monitor_details) 

    #getting the refresh rate

    pattern_refresh_rate = r"\d+\s*[Hh][Zz]"
    refresh_rate = [re.findall(pattern_refresh_rate,specificatie) for specificatie in specificatii]
    refresh_rate = [item for sublist in refresh_rate for item in sublist]
    print('the refresh rate is ')
    print(refresh_rate)
    
    if refresh_rate:
            monitor_details.append(refresh_rate)
            print('the refresh rate was added')
            print(monitor_details)  
    else: 
            refresh_rate = 'not found'
            monitor_details.append(refresh_rate)
            print('the refresh_rate could not be found')
            print(monitor_details) 
            

    #getting the rating and number of reviews
    if monitor.find('a', class_  =  'rating-text gtm_rp101318' ) is None:

        print('No reviews or rating')
        rating = 'No rating'
        monitor_details.append(rating)

        no_of_reviews = 'no reviews'
        monitor_details.append(no_of_reviews)
    
        print(monitor_details)

        
    else:
        review_and_ratings = monitor.find('a', class_  =  'rating-text gtm_rp101318' ).text.splitlines()
        del review_and_ratings[0]
        rating = review_and_ratings[0].strip()
        monitor_details.append(rating)
        print('the rating was added')
        print(monitor_details)

        no_of_reviews = review_and_ratings[1].strip()
        monitor_details.append(no_of_reviews)
        print('the number of reviews  was added')
        print(monitor_details)

    #getting the code

    if monitor.find('span' , class_="product-code-display hidden-xs") is None:
        print('No code found')
        cod_produs = 'No code'
        monitor_details.append(cod_produs)    
        print(monitor_details)
    else:
        cod_produs = monitor.find('span' , class_="product-code-display hidden-xs")
        cod_produs = cod_produs.text.strip().split()[2]
        monitor_details.append(cod_produs)
        print('the product code was added')
        print(monitor_details)

   
    return(monitor_details)

    

    

In [ ]:
    header = pd.DataFrame(columns = coloane)
    header

In [ ]:
test_soup.find_all('li')

In [ ]:
link_emag = f'https://www.emag.ro/monitoare-lcd-led/p89/c'
pagina_monitoare = requests.get(url, headers = HEADERS)
pagina_monitoare

In [ ]:
import math
numar_de_produse = soup.find('span' , class_ = 'title-phrasing title-phrasing-sm').text
numar_de_produse = int( numar_de_produse[:4])
numar_de_pagini = math.ceil(numar_de_produse / 60)
numar_de_pagini  
for x in range(2,numar_de_pagini):
    print(x)

## Getting the monitors script

In [ ]:
coloane = ['Nume' , 'Firma'  , 'Pret' , 'Dimensiune (Inci)' , 'Rezolutie' , 'Refresh rate' , 'Rating', 'Nr Review','Cod produs']
header = pd.DataFrame(columns = coloane)
header
from termcolor import colored

for x in range(2,numar_de_pagini+1):
    link_emag = f'https://www.emag.ro/monitoare-lcd-led/p{x}/c'
    print(colored('Link emag este '+ link_emag , 'green' ) )
    pagina_monitoare = requests.get(link_emag, headers = HEADERS)
    
    soup = BeautifulSoup(pagina_monitoare.content , 'html.parser')
    link_list = find_monitor_links(soup)
    link_list = set(link_list)
    non_qm_link_list = []
    for link in link_list:
        if link[-1] != '?':
            non_qm_link_list.append(link)

    for monitor in non_qm_link_list:
        print(monitor)
        produs_de_test = requests.get(monitor, headers = HEADERS)
        monitor_soup = BeautifulSoup(produs_de_test.content , 'html.parser')
        monitor_details = get_monitor_information(monitor_soup)
        length = len(header)
        header.loc[length] = monitor_details

      

In [ ]:
header

In [ ]:
header.to_excel(r"C:\panda_folder_files\monitoare_emag_data_3.xlsx", index = False)